# Heuristic V1

In [6]:
import pickle
import numpy as np
import StrokeDef
import xmlparse
import xmltodict
import score_strokes

from StrokeDef import loadStrokeDict

"""

"""

'\n\n'

## Comparing Gene Strokes to Archetypal Strokes

In [7]:
archetypal_strokes = loadStrokeDict()

"""

Parameters:
    gene - .gene file
    OR list of strokes, total num of strokes

Return Value:
    total_array - an array containing the total percentages
    
    percent_arrays - a 2D array
"""
#def getGeneStrokePercentages():

    # char_data = loadGeometryBases(data_dir)



"""
Gives a list of scores for how well a given gene stroke matches each of the archetypal strokes

Parameters:
    geneStroke - the gene stroke to be compared to all the archetypal strokes
    
Return Value:
    strokePercentages - an array containing 51 elements, each representing how well geneStroke
    compares to each of the archetypal strokes
"""
def getStrokePercentage(g_points, g_frac_prog):
    
    errors = [] # need to convert errors to scores, what is maximum error?  
    for key in archetypal_strokes:
        points = archetypal_strokes[key].arial
        frac_prog = getFractionalProgress(points)
        error = strokeErrorScaled(g_points, g_frac_prog, points, frac_prog)
        errors.append(error)

# unpickle, figure out the format of StrokeType object
# get the p_stroke and p_ref necessary for strokeError
# use strokeErrorScaled?


"""
"""
def getFractionalProgress(point_list):
    dists = [0]
    for i in range(len(point_list)-1):
        dists.append((point_list[i]-point_list[i+1])+dists[-1])
    dists = np.array(dists)
    dists /= dists.max()
    return dists


"""
*Might already be a pre-existing function for this*
Makes it so that the elements of a given list sum to 1,
by dividing each element by the sum of all the elements

Parameters:
    list - list to be manipulated

Return value
    stand_list - list whose elements sum to 1
"""
#def sumToOne()

FileNotFoundError: [Errno 2] No such file or directory: '../Data/Strokes'

## Comparing Gene Stroke Percentage to Archetype Stroke Counts

In [ ]:
"""
Compares the stroke percentages of a gene to the stroke counts of every archetype in order to
find the top 100 archetypes that most closely resemble the gene

Parameters:
    geneStrokePercents - an array containing the total percentages
    corresponding to the number of archetypal strokes present in the gene
    
    numStrokes - number of strokes contained in the gene

Return Value:
    archetype_matches - a list of the top 50 archetypes whose stroke counts most closely match the gene's stroke percentages

"""
def strokeCountComparison()

## Check Archetypes for Highest Score

In [ ]:
"""
IDEAS:
"""

In [ ]:
# Test
stroke_name = "H1"
archetypal_strokes = loadStrokeDict()
points = archetypal_strokes[stroke_name]